In [20]:
import random
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Mecab
from googletrans import Translator

In [23]:
# Back Translation Function
def back_translation(text):
    """
    Perform back translation (Korean → English → Korean) for text augmentation.
    """
    translator = Translator()
    try:
        translated = translator.translate(text, src='ko', dest='en').text
        return translator.translate(translated, src='en', dest='ko').text
    except Exception as e:
        print(f"Back translation error: {e}")
        return text  # Fallback to original text if error occurs

# Random Deletion
def random_deletion(sentence, p=0.1):
    words = sentence.split()
    if len(words) == 1:
        return sentence
    return ' '.join([word for word in words if random.uniform(0, 1) > p])

# Random Word Swap
def random_swap(sentence, n=1):
    words = sentence.split()
    for _ in range(n):
        if len(words) > 1:
            idx1, idx2 = random.sample(range(len(words)), 2)
            words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

# Random Insertion
def random_insertion(sentence, n=1):
    words = sentence.split()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        words.insert(idx, words[idx])
    return ' '.join(words)

# AEDA (Punctuation Insertion)
def aeda_augmentation(sentence, num_punct=1):
    punctuations = ['.', ',', '!', '?', ';']
    words = sentence.split()
    for _ in range(num_punct):
        idx = random.randint(0, len(words))
        punct = random.choice(punctuations)
        words.insert(idx, punct)
    return ' '.join(words)

# Augmentation Function
def augment_text_data(text_col, repetition=1):
    """
    Apply text augmentations (EDA, AEDA, Back-Translation) for each input text.
    
    Ensures output length matches input length by selecting only one augmented version per text.
    """
    augmented_data = []
    for text in tqdm(text_col, desc="Augmenting Text"):
        # Store augmented versions
        augmented_versions = []
        
        for _ in range(repetition):
            try:
                eda_augmented = random_deletion(text, p=0.1)
                eda_augmented = random_swap(eda_augmented, n=1)
                eda_augmented = random_insertion(eda_augmented, n=1)

                aeda_augmented = aeda_augmentation(text, num_punct=1)
                back_translated = back_translation(text)

                # Collect augmented versions
                augmented_versions.extend([eda_augmented, aeda_augmented, back_translated])
            except Exception as e:
                print(f"Augmentation error: {e}")
                augmented_versions.append(text)  # Fallback to original text
        
        # Select one augmented version (e.g., the first) to maintain alignment
        augmented_data.append(augmented_versions[0])
    
    return augmented_data


In [24]:
back_translation("""
    #Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.",건강검진 받기
    """)

'#Person1#: 안녕하세요, Smith. 저는 Hawkins 의사입니다. 오늘 왜 왔습니까?\n#person2#: 건강 검진을받는 것이 좋은 생각이라고 생각합니다.\n#person1#: 예, 5 년 동안 건강 수표를받지 못했습니다. 매년 가져와야합니다.\n#person2#: 알고 있습니다. 그러나 문제가 없다면 왜 의사를 만나러 가야합니까?\n#person1#: 심각한 질병을 피하는 가장 좋은 방법은 일찍 찾는 것입니다. 따라서 건강을 위해 매년 한 번 이상 오십시오.\n#person2#: 좋아요.\n#person1#: 여기. 눈과 귀가 잘 보입니다.\n#person2#: 예.\n#person1#: 아시다시피, 담배는 폐암과 심장병의 주요 원인입니다.\n#Person2#: 수백 번 시도했지만 습관을 버리는 것은 어렵습니다.\n#person1#: 우리는 도움이 될 수있는 수업과 약물을 제공합니다. 외출하기 전에 더 많은 정보를 제공 할 것입니다.\n#person2#: 좋아요, 감사합니다, 의사.","Smith 씨는 건강 점검을 받고 있으며 Hawkins는 매년 건강 검진을받을 것을 권장합니다. Hawkins는 Smith 씨가 흡연을 중단 할 수있는 수업 및 약물에 대한 정보를 제공 할 것입니다."'

In [ ]:
"스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 
호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.",건강검진 받기,
"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요? 번 건강검진을 받는 것이 좋을 것 같아서요. 
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 매년 받아야 합니다. 
#Person2#: 알고 하지만 아무 문제가 없다면 왜 의사를 가야 하나요? 
#Person1#: 심각한 질병을 피하는 가장 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 
#Person2#: 알겠습니다. #Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 네. 
#Person1#: 당신도 담배는 폐암과 심장병의 주요 원인입니다. 끊으셔야 합니다. 
#Person2#: 수백 #Person2#: 시도했지만, 습관을 버리는 것이 어렵습니다. 
#Person1#: 우리는 도움이 될 수 있는 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다. 
# #Person2#: 알겠습니다, 감사합니다, 의사선생님."
train_7354,"#Person1#: 아빠, 숙제 다 했어요.

In [ ]:
import random

# Load the data from train.csv
train_data = pd.read_csv('/root/NLP/data/train.csv')

# Sample a smaller subset of the data (e.g., 1/3 of the original data)
subset_size = int(len(train_data) / 3)  # For approximately 2 hours of processing
train_sample = train_data.sample(n=subset_size, random_state=42)

# Apply augmentation to the sampled 'dialogue' column (optional: reduce repetition if needed)
train_sample['augmented'] = augment_text_data(train_sample['dialogue'], repetition=1)

# Save the augmented data to a new file
train_sample.to_csv("/root/NLP/data/aug_train.csv", index=False)

print("Data augmentation complete!")

Augmenting Text: 100%|██████████| 4152/4152 [1:20:35<00:00,  1.16s/it]

Data augmentation complete!


In [ ]:
import random

# Load the data from train.csv
train_data = pd.read_csv('/root/NLP/data/train.csv')

# Sample a smaller subset of the data (e.g., 1/3 of the original data)
subset_size = int(len(train_data) / 3)  # For approximately 2 hours of processing
train_sample = train_data.sample(n=subset_size, random_state=42)

# Apply augmentation to the sampled 'dialogue' column (optional: reduce repetition if needed)
train_sample['dialogue'] = augment_text_data(train_sample['dialogue'], repetition=1)

# Save the augmented data to a new file
train_sample.to_csv("/root/NLP/data/aug_train.csv", index=False)

print("Data augmentation complete!")

In [ ]:
# 열 순서
# 

In [ ]:
# We'll keep the format consistent with fname, dialogue, summary, topic
augmented_data = train_sample[['fname', 'dialogue', 'summary', 'topic']]  # Keep the same format

# Combine the original train data and the augmented data (including augmented dialogue)
combined_data = pd.concat([train_data[['fname', 'dialogue', 'summary', 'topic']], 
                           augmented_data[['fname', 'dialogue', 'summary', 'topic']]], 
                          axis=0, ignore_index=True)

# Save the combined data (it will include original and augmented data)
combined_data.to_csv("/root/NLP/data/combined_train.csv", index=False)

print("Data augmentation and combination complete!")

Data augmentation and combination complete!


In [16]:
train_dialog_length = train_sample['dialogue'].apply(lambda x:len(x))

print(train_dialog_length.describe())

count    4152.000000
mean      440.151734
std       225.150379
min       103.000000
25%       297.000000
50%       397.000000
75%       541.000000
max      2339.000000
Name: dialogue, dtype: float64


In [ ]:
# Preview the first few rows to see the data content
print(combined_data.head())

# Check dataset size
num_rows, num_columns = augmented_data.shape
print(f"Number of rows (sentences): {num_rows}")
print(f"Number of columns: {num_columns}")

     fname                                           dialogue  \
0  train_0  #Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...   
1  train_1  #Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...   
2  train_2  #Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...   
3  train_3  #Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...   
4  train_4  #Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...   

                                             summary     topic  
0  스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...   건강검진 받기  
1  파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...        백신  
2  #Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...     열쇠 찾기  
3  #Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...  여자친구가 있다  
4  말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...        댄스  
Number of rows (sentences): 4152
Number of columns: 4
